In [1]:
# Convenient jupyter setup
%load_ext autoreload
%autoreload 2

!pip list | grep epi_to_express  # Make sure you have the project installed as editable library

# if you do not have it installed yet, run
#import os
#os.chdir('/rds/general/project/neurogenomics-lab/live/Projects/epi_to_express') #need to change to parent directory..
#!pip install -e .
# in the parent directory of the model

In [1]:
"""Main module to load and train the model. This should be the program entry point."""
#generic imports
import os
import pathlib
import random
from datetime import datetime
import time
import numpy as np
import pandas as pd
import math
import argparse
import itertools

#import constants
from epi_to_express.constants import (
    CHROM_LEN, 
    CHROMOSOMES, 
    SAMPLES,
    SAMPLE_IDS,
    CHROMOSOME_DATA,
    SRC_PATH,
    ASSAYS,
    PROJECT_PATH)

#model imports
import tensorflow as tf
#data loading imports
from epi_to_express.utils import Roadmap3D_tf
from epi_to_express.model import conv_profile_task_base

#pass inputs
# argv
#def get_args():
#    parser = argparse.ArgumentParser(description="train")
#    parser.add_argument('-c', '--CELL', default='', type=str, help='Cell to train in')
#    parser.add_argument('-m', '--MARK', default='', type=str, help='Mark to train on')
#    args = parser.parse_args()
#    return args

#args=get_args()

CELL='E003'
#leading and trailing whitespace
CELL=CELL.strip()
#assert it's a valid choice
assert CELL in SAMPLE_IDS, f"{CELL} not valid. Must choose valid cell: {SAMPLE_IDS}"

MARK='h3k4me1'
MARK=MARK.strip()
#assert it's a valid choice
assert MARK in ASSAYS, f"{MARK} not valid. Must choose valid assay: {ASSAYS}"

print("---------------------------------")
print(CELL)
print(MARK)
print("---------------------------------")

# Set random seeds.
np.random.seed(101)
tf.random.set_seed(101)
random.seed(101)
    
SAVE_PATH = pathlib.Path("./model_results")
SAVE_PATH.mkdir(parents=True, exist_ok=True)

MOD_SAVE_PATH = pathlib.Path("./model_results/models")
MOD_SAVE_PATH.mkdir(parents=True, exist_ok=True)

# 1. --- SETUP PARAMETERS ------------------------------------------------

#what will be used to predict expression
features = [MARK]
#what cell will we predict in
cell = CELL
#resolution for training assay
pred_resolution = 100# choice of 100, 500, 2000
# 1 Mb of the assay will be considered for the prediction of gene expression
window_size = 20_000
#number of k-fold cross validation
k_fold = 4
#seed
seed = 123
#regression problem
y_type = 'log2RPKM'

# Model specifics
batch_size = 64
n_epochs = 100
init_learning_rate = 0.001
lr_decay_factor = 0.2
lr_patience = 3
es_patience = 12

# 2. --- Dataset parameters -------------------------------
train_dir = PROJECT_PATH/'chromoformer'/'preprocessing'
train_meta = train_dir / 'train.csv'
meta = pd.read_csv(train_meta) \
    .sample(frac=1, random_state=seed) \
    .reset_index(drop=True) # load and shuffle.

#filter metadat to cell type of interest
meta = meta[meta.eid == CELL]

# Split genes into two sets (train/val).
genes = set(meta.gene_id.unique())
n_genes = len(genes)
print('Target genes:', len(genes))

#get data for folds separated
qs = [
    meta[meta.split == 1].gene_id.tolist(),
    meta[meta.split == 2].gene_id.tolist(),
    meta[meta.split == 3].gene_id.tolist(),
    meta[meta.split == 4].gene_id.tolist(),
]

# 3. --- Train models -------------------------------
# loop through each fold
for ind,fold in enumerate([x+1 for x in range(k_fold)]):
    print(f"K-fold Cross-Validation - blind test: {fold}")
    #get fold specific data ----
    train_genes = qs[(fold + 0) % 4] + qs[(fold + 1) % 4] + qs[(fold + 2) % 4]
    val_genes = qs[(fold + 3) % 4]
    
    train_genes = train_genes[1:100]
    val_genes = val_genes[1:100]
    
    #split val_genes in two to get validation and test set
    # train/val split by chrom so do the same for val test
    val_test_genes = val_genes
    val_test_chrom = list(set(meta[meta.gene_id.isin(val_test_genes)]['chrom']))
    val_chrom = val_test_chrom[0:len(val_test_chrom)//2]
    test_chrom = val_test_chrom[len(val_test_chrom)//2:len(val_test_chrom)]
    val_genes = meta[meta.gene_id.isin(val_test_genes) & meta.chrom.isin(val_chrom)]['gene_id'].tolist()
    test_genes = meta[meta.gene_id.isin(val_test_genes) & meta.chrom.isin(test_chrom)]['gene_id'].tolist()
    #----
    #data loaders ----
    training_generator = Roadmap3D_tf(cell, train_genes, batch_size=batch_size,
                                      w_prom=window_size, w_max=window_size,
                                      marks = features,y_type=y_type,
                                      pred_res = pred_resolution,
                                      return_pcres=False)
    validation_generator = Roadmap3D_tf(cell, val_genes, batch_size=batch_size,
                                        w_prom=window_size, w_max=window_size,
                                        marks = features,y_type=y_type,
                                        pred_res = pred_resolution,
                                        return_pcres=False)
    #----
    #train ----
    print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))    

    # import conv model
    model = conv_profile_task_base(output_shape=[1,1],window_size=window_size,
                                   pred_res=pred_resolution)

    #learning rate schedule
    lr_schedule = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", 
                                                     factor=lr_decay_factor, 
                                                     patience=lr_patience)
    #early stopping
    es = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=es_patience,
                                          #save best weights
                                          restore_best_weights=True)

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=init_learning_rate),
                  loss=tf.keras.losses.mean_squared_error,
                  metrics='mse')

    # Train model on dataset
    model.fit(training_generator,
              validation_data=validation_generator,
              epochs=n_epochs,
              verbose=2,
              use_multiprocessing=False,#started getting errors when set to True...
              callbacks=[es,lr_schedule]
             )

    model.save(f"{MOD_SAVE_PATH}/mod_{'-'.join(cell)}_{'-'.join(features)}_kfold{ind}")



2022-12-08 18:08:44.133484: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-08 18:08:44.133516: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


---------------------------------
E003
h3k4me1
---------------------------------
Target genes: 18955
K-fold Cross-Validation - blind test: 1
Num GPUs Available:  0


2022-12-08 18:08:51.317948: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-12-08 18:08:51.317979: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (workstation-neurogenomics): /proc/driver/nvidia/version does not exist
2022-12-08 18:08:51.328492: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/100


1/1 - 6s - loss: 144376.0156 - mse: 144376.0156 - lr: 0.0010 - 6s/epoch - 6s/step
Epoch 2/100


...

1/1 - 6s - loss: 162281.1562 - mse: 162281.1562 - lr: 0.0010 - 6s/epoch - 6s/step
Epoch 5/100


1/1 - 4s - loss: 144243.9219 - mse: 144243.9219 - lr: 0.0010 - 4s/epoch - 4s/step
Epoch 6/100


1/1 - 7s - loss: 162287.2969 - mse: 162287.2969 - lr: 0.0010 - 7s/epoch - 7s/step
Epoch 7/100


1/1 - 4s - loss: 144248.1250 - mse: 144248.1250 - lr: 0.0010 - 4s/epoch - 4s/step
Epoch 8/100


KeyboardInterrupt: 

In [2]:
X,y=next(iter(training_generator))

In [3]:
y

array([ 8.4229727e+00, -1.0740000e+03, -4.9213901e+00, -1.0740000e+03,
        3.4715776e+00, -4.7958593e+00, -3.9213711e-01, -5.0588937e+00,
        4.7768412e+00,  4.8452392e+00,  6.4708424e-01, -4.2933588e+00,
        2.8601682e+00, -2.8996952e+00,  9.4385986e+00,  2.5651097e+00,
        4.3173766e+00, -9.9657841e+00,  6.7954164e+00, -1.0740000e+03,
        2.1446991e+00,  2.5489297e+00,  3.4233093e+00,  6.6399703e+00,
       -4.3808217e+00,  1.1503325e-01,  2.4276061e+00, -1.0740000e+03,
       -1.1584294e+00,  2.7398481e+00,  4.5903625e+00, -4.2933588e+00,
       -8.3808222e+00,  2.8483975e+00,  3.2374108e+00, -1.0740000e+03,
       -1.0740000e+03,  9.3407154e+00,  9.6938074e-01, -4.4739313e+00,
        4.7842422e+00,  2.9789279e+00, -1.0740000e+03,  6.2220702e+00,
       -3.4900508e+00,  5.7468233e+00,  5.9101329e+00, -8.9657841e+00,
       -1.4560533e-01,  9.7232503e-01,  2.8397582e+00, -1.0740000e+03,
       -2.4579897e+00,  3.4273379e+00,  3.2873244e+00,  3.8413694e+00,
      

In [4]:
X

{'x_p_pred_res': <tf.Tensor: shape=(64, 200, 1), dtype=float32, numpy=
 array([[[0.        ],
         [0.        ],
         [0.        ],
         ...,
         [0.23901689],
         [0.37156358],
         [0.24686006]],
 
        [[0.        ],
         [0.5423243 ],
         [0.        ],
         ...,
         [0.91629076],
         [0.67803353],
         [0.8329091 ]],
 
        [[1.2612978 ],
         [0.7975072 ],
         [0.08617773],
         ...,
         [0.        ],
         [0.35767448],
         [0.2546422 ]],
 
        ...,
 
        [[0.30748472],
         [0.37156358],
         [0.48858002],
         ...,
         [1.5769148 ],
         [1.4973885 ],
         [1.0647107 ]],
 
        [[0.        ],
         [0.5423243 ],
         [0.20701419],
         ...,
         [0.7975072 ],
         [0.79299253],
         [1.4770488 ]],
 
        [[0.73716414],
         [0.60431594],
         [0.8415673 ],
         ...,
         [0.5423243 ],
         [0.5423243 ],
         [